In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import textwrap
import json
import os

In [ ]:
data_weather = pd.read_csv('../data/raw/weather/weather.csv', sep = "\t")
data_corona_de = pd.read_csv("../data/raw/corona/de_corona.csv", sep = "\t")

In [ ]:
print(data_weather.shape)
data_weather.head()

In [ ]:
print(list(data_weather.columns))
np.unique(data_weather["iso3166-2"])

Sanity check

In [ ]:
count = 0
for i in data_weather:
    #print(sum(pd.isnull(data_weather[i])))
    if sum(pd.isnull(data_weather[i])) != 0:
        count += 1
print(count)

Filtering for germany

In [ ]:
de_codes = ['DE-BB', 'DE-BE', 'DE-BW', 'DE-BY', 'DE-HB', 'DE-HE', 'DE-HH', 'DE-MV', 'DE-NI', 'DE-NW', 
            'DE-RP', 'DE-SH', 'DE-SL', 'DE-SN', 'DE-ST', 'DE-TH']
mask = np.isin(data_weather, de_codes)
data_weather_de = data_weather[mask]

print(data_weather_de.shape)
np.unique(data_weather_de["iso3166-2"])

In [ ]:
# saving the filtered dataset for germany
def saving_csv(file, name):
    file.to_csv(f'../data/interim/weather_{name}.csv')

In [ ]:
saving_csv(data_weather_de, "germany")

In [ ]:
print(data_corona_de.shape)
data_corona_de.head()

# min, mean, median, and max of regions

In [ ]:
# masking for different regions
for codes, names in zip(de_codes, de_codes):
    mask = np.isin(data_weather_de, codes)
    region_data = data_weather_de[mask]
    saving_csv(region_data, names)

In [ ]:
# filter for numerical columns, they are from index 2 and onwards
for region in de_codes:
    with open(f'../data/interim/weather_{region}.csv', "o") as region_data:
        for i in range(2, len(list(data_weather_de))):
            data_weather_de.iloc[:, i]


In [ ]:
len(list(data_weather_de))